In [1]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import os

In [2]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [3]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name'])

mycursor = mydb.cursor()

In [4]:

query = "select userid from usr_applied_jobs_info where unixtimestamp>1707374166"

mycursor.execute(query)
result = mycursor.fetchall()

In [5]:
active_users = pd.DataFrame(result,columns=['userid'])

In [6]:
active_users.drop_duplicates(inplace=True)

In [7]:
active_users

,userid
0,1780851
1,1614881
2,939714
3,1684573
4,2142750
...,...
1286439,1038367
1286456,2821255
1286463,541092
1286541,2587238


In [8]:
ulist = tuple(active_users['userid'].tolist())

In [9]:

query = f"select id, resume_last_updated, resume_first_uploaded from profile_info where id in {ulist}"

mycursor.execute(query)
result = mycursor.fetchall()

In [13]:
rdata = pd.DataFrame(result,columns=['id','last_upload','first_upload'])

In [20]:
rdata['first_upload'].drop('0000-00-00 00:00:00') 

KeyError: "['0000-00-00 00:00:00'] not found in axis"

In [21]:
rdata['first_upload'] = pd.to_datetime(rdata['first_upload'],errors='coerce')
rdata['last_upload'] = pd.to_datetime(rdata['last_upload'])

# Calculate the difference in years
rdata['year_difference'] = rdata['last_upload'] - rdata['first_upload']
rdata['year_difference'] = rdata['year_difference'] / pd.Timedelta(days=365.25)

# To round the years to a sensible number of decimal places, e.g., 2 decimal places
rdata['year_difference'] = rdata['year_difference'].round(2)


In [22]:
select = rdata[rdata['year_difference']>3]

In [27]:
select

,id,last_upload,first_upload,year_difference
0,6,2023-06-08 15:52:56,2013-10-01 23:16:54,9.68
1,17,2024-01-19 12:32:39,2013-06-25 14:21:21,10.57
2,77,2024-01-05 12:05:06,2013-01-26 11:07:45,10.94
3,79,2024-01-21 16:58:45,2013-09-27 12:03:50,10.32
4,92,2023-12-25 21:28:18,2012-10-19 05:27:47,11.18
...,...,...,...,...
51468,1982216,2024-03-05 23:16:25,2021-03-02 14:58:51,3.01
51494,1982834,2024-03-06 22:56:14,2021-03-03 08:06:48,3.01
51498,1983003,2024-03-08 05:07:37,2021-03-03 12:26:40,3.01
51499,1983101,2024-03-06 11:45:47,2021-03-03 13:53:33,3.01


In [28]:
from user_info import get_user_info

In [29]:
r_text = {}
for uid in tqdm(select['id'].tolist()):
    response = get_user_info(uid)
    try:
        r_text[uid] = response['resume'][0]
    except:
        pass

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45170/45170 [06:32<00:00, 115.06it/s]


In [30]:
import json
with open('../custom_data/relevant_users_feb24.json','w') as f:
    json.dump(r_text,f)